In [1]:
## S&P 500 STOCK CLUSTERING AND NEURAL NETWORK ANALYSIS ##
# This script downloads the most recent listing of stock tickers of the S&P 500 from Wikipedia
# It then downloads the basic stock prices dating back to January 1st, 2015
# These data points are aggreagated into basic average summaries as well as processed for neural network analysis.
# At its most fundimental level, the algorithm uses the previous n days of adjusted close data to predict a rise
# or fall in price the following day/week/month.

# It is important to note that the S&P 500 does not mimic the behavior of all stocks. Separately, the metrics by which
# we evaluate our model are constantly improving. At this point, there is a theoretical lower-bound of 50% accuracy:
# did the stock go up or down. However we also seek to gauge how much the stock increased or decreased by increasing
# the number of neurons in the output layer or by running an entirely separate anlaysis--such a discussion is ongoing.

In [2]:
import datetime
import urllib.request
import pandas as pd
from pandas import DataFrame
from pandas.io.data import DataReader
import re

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pandas/io/data.py:35: FutureWarning: 
The pandas.io.data module is moved to a separate package (pandas-datareader) and will be removed from pandas in a future version.
After installing the pandas-datareader package (https://github.com/pydata/pandas-datareader), you can change the import ``from pandas.io import data, wb`` to ``from pandas_datareader import data, wb``.
  FutureWarning)


In [3]:
# COLLECT STOCK TICKERS
wiki_page = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
wiki_page = str(wiki_page.read())

In [4]:
ticker_locations_part_one = [m.start() for m in re.finditer('XNYS', wiki_page)]
ticker_locations_part_two = [m.start() for m in re.finditer('.com/symbol/', wiki_page)]
all_tickers = []
for t in range(0,len(ticker_locations_part_one)): # t=1
    ticker = wiki_page[ticker_locations_part_one[t]:ticker_locations_part_one[t]+15]
    ticker = re.search('XNYS:(.+?)>',ticker).group(1).replace('"','')
    all_tickers.append(ticker)
for t in range(0,len(ticker_locations_part_two)):
    ticker = wiki_page[ticker_locations_part_two[t]:ticker_locations_part_two[t]+25]
    ticker = re.search('com/symbol/(.+?)>',ticker).group(1).replace('"','')
    all_tickers.append(ticker)

In [11]:
symbols_list = all_tickers

symbols=[]
for ticker in symbols_list:
    try:
        r = DataReader(ticker, "yahoo", start=datetime.datetime(2014, 12, 30))
        # add a symbol column
        r['Symbol'] = ticker 
        symbols.append(r)
    except:
        print(ticker, "has been renamed")
# concatenate all the dfs
df = pd.concat(symbols)
#define cell with the columns that i need
cell= df[['Symbol','Open','High','Low','Adj Close','Volume']]
#changing sort of Symbol (ascending) and Date(descending) setting Symbol as first column and changing date format
cell = cell.reset_index().sort(['Symbol', 'Date'], ascending=[1,0]).set_index(['Symbol'])

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:17: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [6]:
last_close = []
five_day_avg = []
one_month_avg = []
two_month_avg = []
three_month_avg = []
six_month_avg = []
twelve_month_avg = []

last_close_vol = []
five_day_avg_vol = []
one_month_avg_vol = []
two_month_avg_vol = []
three_month_avg_vol = []
six_month_avg_vol = []
twelve_month_avg_vol = []

for i in range(0,len(symbols_list)):
    ticker = symbols_list[i]
    subset = cell.loc[ticker]

    last_close.append(subset.iloc[2,:]["Adj Close"])
    five_day_avg.append(subset.iloc[2:7,:]["Adj Close"].mean())
    one_month_avg.append(subset.iloc[2:32,:]["Adj Close"].mean())
    two_month_avg.append(subset.iloc[2:62,:]["Adj Close"].mean())
    three_month_avg.append(subset.iloc[2:92,:]["Adj Close"].mean())
    six_month_avg.append(subset.iloc[2:182,:]["Adj Close"].mean())
    twelve_month_avg.append(subset.iloc[2:365,:]["Adj Close"].mean())

    last_close_vol.append(subset.iloc[2,:]["Volume"])
    five_day_avg_vol.append(subset.iloc[2:7,:]["Volume"].mean())
    one_month_avg_vol.append(subset.iloc[2:32,:]["Volume"].mean())
    two_month_avg_vol.append(subset.iloc[2:62,:]["Volume"].mean())
    three_month_avg_vol.append(subset.iloc[2:92,:]["Volume"].mean())
    six_month_avg_vol.append(subset.iloc[2:182,:]["Volume"].mean())
    twelve_month_avg_vol.append(subset.iloc[2:365,:]["Volume"].mean())

ticker_data = {'Ticker':ticker,'Last Close':last_close,'Five Day Avg':five_day_avg,'One Month Avg':one_month_avg,
               'Two Month Avg':two_month_avg,'Three Month Avg':three_month_avg,'Six Month Avg':six_month_avg,
               'Twelve Month Avg':twelve_month_avg,'Last Close Vol':last_close_vol,'Five Day Avg Vol':five_day_avg_vol,
               'One Month Avg Vol':one_month_avg_vol,'Two Month Avg Vol':two_month_avg_vol,
               'Three Month Avg Vol':three_month_avg_vol,'Six Month Avg Vol':six_month_avg_vol,
               'Twelve Month Avg Vol':one_month_avg_vol}
ticker_outputs = DataFrame(data=ticker_data, index=symbols_list)
ticker_outputs = ticker_outputs[['Last Close','Five Day Avg','One Month Avg','Two Month Avg','Three Month Avg',
                                'Six Month Avg','Twelve Month Avg','Last Close Vol','Five Day Avg Vol',
                                'One Month Avg Vol','Two Month Avg Vol','Three Month Avg Vol','Six Month Avg Vol',
                                'Twelve Month Avg Vol']]

In [7]:
ticker_outputs

,Last Close,Five Day Avg,One Month Avg,Two Month Avg,Three Month Avg,Six Month Avg,Twelve Month Avg,Last Close Vol,Five Day Avg Vol,One Month Avg Vol,Two Month Avg Vol,Three Month Avg Vol,Six Month Avg Vol,Twelve Month Avg Vol
MMM,166.509995,168.673999,173.021666,175.891757,175.777333,169.635160,158.132972,2174900,2378660.0,2.009733e+06,1.691380e+06,1.808380e+06,1.889524e+06,2.009733e+06
ABT,39.959999,40.360000,41.395425,42.272396,41.869619,40.559100,42.296000,7196700,7359240.0,8.158663e+06,8.779402e+06,9.504498e+06,9.189864e+06,8.158663e+06
ABBV,61.020000,61.285999,62.261819,63.564528,63.177034,60.444267,59.648992,4560800,4732280.0,5.583167e+06,5.999667e+06,6.589789e+06,7.366378e+06,5.583167e+06
ACN,115.449997,115.631999,115.049856,113.760941,113.472940,111.796965,105.183637,1830900,1810800.0,2.789107e+06,2.353807e+06,2.424213e+06,2.395741e+06,2.789107e+06
AYI,231.339996,237.393997,250.399959,260.935830,259.790377,247.212562,223.843097,379800,437000.0,4.950967e+05,3.533700e+05,3.510278e+05,4.513339e+05,4.950967e+05
AAP,143.960007,142.420001,146.595560,153.559423,156.550769,155.100263,161.127517,871500,885600.0,9.378767e+05,1.044087e+06,1.009377e+06,9.823489e+05,9.378767e+05
AES,11.888325,11.916065,12.177277,12.175296,12.149235,11.417362,11.016202,3757900,3747380.0,5.471703e+06,5.085715e+06,5.005650e+06,4.998976e+06,5.471703e+06
AET,111.010002,111.370000,113.255774,115.398357,116.138517,113.904163,112.391001,2823800,2216200.0,2.059197e+06,1.874072e+06,2.501471e+06,2.415749e+06,2.059197e+06
AFL,69.800003,69.720000,71.419333,72.092247,71.959075,68.344096,63.853359,1206900,1477140.0,1.550340e+06,1.592313e+06,1.745459e+06,1.974711e+06,1.550340e+06
AMG,140.979996,141.562000,144.692667,143.947334,143.704889,151.127111,166.978953,340500,409360.0,4.636600e+05,4.280850e+05,5.069700e+05,5.368167e+05,4.636600e+05


In [12]:
ticker_outputs.loc['aapl']

Last Close              1.155900e+02
Five Day Avg            1.170300e+02
One Month Avg           1.150593e+02
Two Month Avg           1.113552e+02
Three Month Avg         1.067081e+02
Six Month Avg           1.031709e+02
Twelve Month Avg        1.079531e+02
Last Close Vol          6.613420e+07
Five Day Avg Vol        3.702408e+07
One Month Avg Vol       3.718071e+07
Two Month Avg Vol       3.523718e+07
Three Month Avg Vol     3.558460e+07
Six Month Avg Vol       3.679348e+07
Twelve Month Avg Vol    3.718071e+07
Name: aapl, dtype: float64

In [13]:
cell.loc['A']

,Date,Open,High,Low,Adj Close,Volume
Symbol,,,,,,
A,2016-10-28,43.380001,43.869999,43.110001,43.220001,1470500
A,2016-10-27,43.970001,44.189999,43.349998,43.380001,2392800
A,2016-10-26,43.419998,43.820000,43.240002,43.369999,2146200
A,2016-10-25,45.259998,45.590000,43.169998,43.509998,3949500
A,2016-10-24,45.880001,45.950001,45.639999,45.849998,1666400
A,2016-10-21,45.669998,45.669998,45.290001,45.570000,1011300
A,2016-10-20,46.000000,46.220001,45.650002,46.049999,1493600
A,2016-10-19,45.410000,46.150002,45.340000,46.020000,1769000
A,2016-10-18,45.700001,45.830002,45.270000,45.439999,1949100
